### <font color=black> IMPORT:: <font color=blue> **py modules** </font> </font>

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from mpl_toolkits.basemap import Basemap
import glob
import os
import shutil
import re
import cartopy
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
%matplotlib inline
import fnmatch
#
PD=os.getcwd(); #PD
#
import fun_pbox as fn

In [2]:
ssp=["ssp119","ssp126","ssp245","ssp370","ssp585"]
#
workflow = ["wf_1e","wf_1f","wf_2e","wf_2f","wf_3e", "wf_3f", "wf_4"]
#
import fun_wfFiles as wfF

---
## <font color=red> Generate:: 001_workflows  </font>
---

In [3]:
# components = wfF.total["wf_1e"]
# components = wfF.component["wf_1e"]
expFolder="/projects/kopp/facts-experiments/221217/"
Workflow_path = fn.FOcr8("001_workflows")
for indv_workflow0,indv_workflow1 in enumerate(workflow):
    wf_path=fn.FOcr8_1(Workflow_path, indv_workflow1) #create wf folder
    for ssp0,ssp1 in enumerate(ssp):
        # ..........................................................................................
        if indv_workflow1 in ['wf_3e', 'wf_3f'] and ssp1 in['ssp119','ssp370']: continue
        if indv_workflow1 in ['wf_4'] and ssp1 in['ssp119','ssp245','ssp370']: continue
        # ..........................................................................................
        ssp_path=fn.FOcr8_1(wf_path, ssp1) #create ssp folders within the wf
        #
        # for wfcomp0,wfcomp1 in enumerate(eval(f"{indv_workflow1}")):
        for wfcomp0,wfcomp1 in enumerate(wfF.component[indv_workflow1]):
            sourcefile_path=expFolder+"coupling."+ssp1+"/output"
            shutil.copy2(sourcefile_path+'/coupling.'+ssp1+'.'+wfcomp1, ssp_path) # copy the file to the ssp folder in the workflow

        for wfcomp0,wfcomp1 in enumerate(wfF.total[indv_workflow1]):
            sourcefile_path=expFolder+"coupling."+ssp1+"/output"
            shutil.copy2(sourcefile_path+'/coupling.'+ssp1+'.'+wfcomp1, ssp_path) # copy the file to the ssp folder in the workflow

---
## <font color=red> Generate:: 002_quant_workflows  </font>
---

In [4]:
import convert_samples_to_dist_MOD
#
expFolder="/projects/kopp/facts-experiments/221217/"
# Create a folder
Workflow_path = fn.FOcr8("002_quant_workflows")
#
for indv_workflow0,indv_workflow1 in enumerate(workflow):
    wf_path=fn.FOcr8_1(Workflow_path, indv_workflow1) #create wf folder
    for ssp0,ssp1 in enumerate(ssp):
        # ..........................................................................................
        if indv_workflow1 in ['wf_3e', 'wf_3f'] and ssp1 in['ssp119','ssp370']: continue
        if indv_workflow1 in ['wf_4'] and ssp1 in['ssp119','ssp245','ssp370']: continue
        # ..........................................................................................
        # Component
        ssp_path=fn.FOcr8_1(wf_path, ssp1) #create ssp folders within the wf
        for wfcomp0,wfcomp1 in enumerate(wfF.component[indv_workflow1]):
            #Convert to dist calling the fn directly
            inFile = PD+'/001_workflows/'+indv_workflow1+'/'+ssp1+'/coupling.'+ssp1+'.'+wfcomp1
            outFile = ssp_path+'/coupling.'+ssp1+'.'+wfcomp1
            convert_samples_to_dist_MOD.ConvertSamplesToDist_dev_dev(inFile, outFile)
        # Total
        for wfcomp0,wfcomp1 in enumerate(wfF.total[indv_workflow1]):
            #Convert to dist calling the fn directly
            inFile = PD+'/001_workflows/'+indv_workflow1+'/'+ssp1+'/coupling.'+ssp1+'.'+wfcomp1
            outFile = ssp_path+'/coupling.'+ssp1+'.'+wfcomp1
            convert_samples_to_dist_MOD.ConvertSamplesToDist_dev_dev(inFile, outFile)

---
## <font color=red> Generate:: 003_pbox  </font>
---

In [5]:
import fun_pbDef as pbdef
Pbox = ["pb_1e","pb_1f","pb_2e","pb_2f"]

In [6]:
import generate_pbox_components_dev_MOD
#
expFolder="/projects/kopp/facts-experiments/221217/"
# Create a folder
Pbox_path = fn.FOcr8("003_pbox")
#
for indv_Pbox0,indv_Pbox1 in enumerate(Pbox):
    pb_path=fn.FOcr8_1(Pbox_path, indv_Pbox1) #create wf folder
    for ssp0,ssp1 in enumerate(ssp):
        # ..........................................................................................
        if indv_Pbox1 in ['pb_2e', 'pb_2f'] and ssp1 in['ssp119','ssp245','ssp370']: continue
        # ..........................................................................................
        ssp_path=fn.FOcr8_1(pb_path, ssp1) #create ssp folders within the pb
        #
        if indv_Pbox1 in ['pb_1e','pb_1f']:
            # Copy all WF files.
            srcDIR=PD+"/002_quant_workflows/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # larmipAIS (common to pb_1e & pb_1f)
            srcDIR=PD+"/002_quant_workflows/"+"wf_2e/"+ssp1; fn.cp(srcDIR,ssp_path,"*larmip.AIS*")
            #
            if indv_Pbox1 in ['pb_1e']:
                # AIS Pbox component
                fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
                fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
                generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                # total
                srcDIR=PD+"/002_quant_workflows/"+"wf_2e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
                fname1=fn.fileNAME0(ssp_path,"total.workflow.wf1e");               fname2=fn.fileNAME0(ssp_path,"total.workflow.wf2e")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      outfile=ssp_path+"/total-workflow.nc"
                generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

                #
            if indv_Pbox1 in ['pb_1f']:
                # AIS Pbox component.
                fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
                fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
                generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                # total
                srcDIR=PD+"/002_quant_workflows/"+"wf_2f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
                fname1=fn.fileNAME0(ssp_path,"total.workflow.wf1f");               fname2=fn.fileNAME0(ssp_path,"total.workflow.wf2f")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      outfile=ssp_path+"/total-workflow.nc"
                generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")


        
        if indv_Pbox1 in ['pb_2e']:
            # Copy all WF files.
            srcDIR=PD+"/002_quant_workflows/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # emulandice.AIS
            srcDIR=PD+"/002_quant_workflows/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*emulandice.AIS*")
            srcDIR=PD+"/002_quant_workflows/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/002_quant_workflows/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");
            srcDIR=PD+"/002_quant_workflows/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.glaciers*");
            
            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # Glaciers Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.glaciers")
            fname2=fn.fileNAME0(ssp_path,"ipccar5.glaciers")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"glaciers-pb"+indv_Pbox1.split('_')[1]+"-glaciers-"+ssp1+"_globalsl.nc"
            generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  
            
            # total
            srcDIR=PD+"/002_quant_workflows/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/002_quant_workflows/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/002_quant_workflows/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2e");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1e"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3e"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*glaciers*", "glaciers-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

        if indv_Pbox1 in ['pb_2f']:
            # Copy all WF files.
            srcDIR=PD+"/002_quant_workflows/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            srcDIR=PD+"/002_quant_workflows/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.icesheets_AIS*");
            srcDIR=PD+"/002_quant_workflows/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/002_quant_workflows/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");


            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"GrIS1f.FittedISMIP.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  

            # total
            srcDIR=PD+"/002_quant_workflows/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/002_quant_workflows/"+"wf_3f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/002_quant_workflows/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2f");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1f"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3f"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            generate_pbox_components_dev_MOD.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

---
## <font color=red> Generate:: 006_CI-level_OP_files  </font>
---

In [7]:
# # Copy sh file
# src="/scratch/pk695/FACTS/nzOG-pk695/scripts__pk695/scripts/006_generate_confidence_level_output_files/generate_confidence_output_files.py"
# dst=PD
# shutil.copy2(src, dst)

In [8]:
import fun_generate_CI_op_files as CI 
pboxdir=PD+'/003_pbox'
outdir = fn.FOcr8("004_CI_files")
CI.GenerateConfidenceFiles(pboxdir, outdir)

In [9]:
# xr.open_dataset("/scratch/pk695/FACTS/nzOG-pk695/local-pk695/006_confidence_output_files/medium_confidence/ssp119/GIS_ssp119_medium_confidence_values.nc")
np.arange(2020,2151,10)
# xr.open_dataset("/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/2023-01_NZ_INSAR/001_DEVEL_Pbox/003_pbox/pb_1e/ssp119/icesheets-pb1e-icesheets-ssp119_AIS_globalsl.nc")
xr.open_dataset("/scratch/pk695/FACTS/nzOG-pk695/local-pk695/005_pboxes/pb_1e/ssp119/icesheets-pb1e-icesheets-ssp119_AIS_localsl.nc")

<xarray.Dataset>
Dimensions:           (locations: 7435, years: 9, quantiles: 107)
Coordinates:
  * locations         (locations) int32 2000000000 2000000010 ... 2000074340
  * years             (years) int32 2020 2030 2040 2050 2060 2070 2080 2090 2100
  * quantiles         (quantiles) float64 0.0 0.001 0.005 ... 0.995 0.999 1.0
Data variables:
    lat               (locations) float32 ...
    lon               (locations) float32 ...
    sea_level_change  (quantiles, years, locations) float32 ...
Attributes:
    description:  Pbox generated from a FACTS sea-level change projection wor...
    history:      Created Fri Apr  7 11:46:34 2023
    source:       Generated with files: nzOG.ssp119.emuAIS.emulandice.AIS_loc...